In [23]:
# Imports
import numpy as np
import torch

In [11]:
# Vocabulary
words = open('../datasets/names.txt', 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
# Datasets
xs, ys = [], []
for word in words:
	chs = ['.'] + ['.'] + list(word) + ['.']
	for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
		ix1 = stoi[ch1]
		ix2 = stoi[ch2]
		ix3 = stoi[ch3]
		
		xs.append([ix1, ix2])
		ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

tensor([0, 0])

In [ ]:
# Creating one-hot vectors
one_hot = torch.zeros(xs.shape[0], (len(chars) + 1) ** 2)
print(one_hot.shape)
for i, x in enumerate(xs):
    one_hot[i][x[0] * (len(chars) + 1) + x[1]] = 1

torch.Size([228146, 729])


tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]])